# Tutorial on connecting DeepPavlov bot to Yandex.Alice
In this tutorial we show how to develop a simple bot and connect it to Yandex.Alice. Our bot is able to greet, say goodbye, and answer questions based on an FAQ list. In our case we take typical questions of MIPT entrant as an example. This list consists of 15 questions with corresponding answers with manually generated 4-6 paraphrases for each question. More details on how to build bots with DeepPavlov can be found [here](https://medium.com/deeppavlov).

## Requirements

To connect your bot to Yandex.Alice you need to have a dedicated IP address. You also need to create an ssl certificate and a key (more details in corresponding section).

First, create a virtual env with python3.6 (`source activate py36`). Then download DeepPavlov

In [ ]:
!pip install deeppavlov

## Implementation

Create skills for greeting, saying goodbye, and fallback (in case a bot doesn't have an appropriate answer).

In [1]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill

hello = PatternMatchingSkill(responses=['Привет!', 'Приветик', 'Здравствуйте', 'Добрый день'],
	patterns=['Привет', 'Здравствуйте', 'Добрый день'])
bye = PatternMatchingSkill(responses=['Пока!', 'До свидания, надеюсь смог вам помочь', 'До встречи!'],
	patterns=['До свидания', 'Пока', 'Спасибо за помощь'])
fallback = PatternMatchingSkill(responses=['Я не понял, но могу попробовать ответить на другой вопрос',
	'Я не понял, задайте другой вопрос'])

Download '.csv' table with questions and answers

In [2]:
import pandas as pd
faq_table = pd.read_csv('http://files.deeppavlov.ai/faq/school/faq_school.csv')
# You may also specify path to a local file instead of a link
faq_table[:10]

,Question,Answer
0,"Мы потеряли свой код регистрации, что нам делать?",Вы можете написать нам письмо на почту приемно...
1,"Утерян код регистрации. Подскажите, что с этим...",Вы можете написать нам письмо на почту приемно...
2,Где взять код регистрации?,Вы можете написать нам письмо на почту приемно...
3,Я забыл код регистрации. Как его восстановить?,Вы можете написать нам письмо на почту приемно...
4,Как восстановить код регистрации?,Вы можете написать нам письмо на почту приемно...
5,"При регистрации мы допустили ошибку, как нам п...",Некоторые данные можно поменять через личный к...
6,При регистрации была допущена ошибка. Я хотел ...,Некоторые данные можно поменять через личный к...
7,"Нужно исправить даннные, что делать?",Некоторые данные можно поменять через личный к...
8,Я ошиблась в имени ребенка. Где можно его изме...,Некоторые данные можно поменять через личный к...
9,Как изменить регистрационные данные?,Некоторые данные можно поменять через личный к...


There is a skill, that can recognise users intent and respond with corresponding message. It's perfectly suitable for an FAQ task. It has following arguments:
1. `data_path` - path/link to '.csv' table
2. `x_col_name` - name of the questions column in the table
3. `y_col_name` - name of the answers column in the table
4. `edit_dict` - dict of edits in config file of skill
5. `save_load_path` - path, where models will be saved or loaded from
6. `train` - should model be trained or not

In [3]:
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill

faq = SimilarityMatchingSkill(data_path='http://files.deeppavlov.ai/faq/school/faq_school.csv',
                          x_col_name='Question', y_col_name='Answer',
                          save_load_path='./faq_school', train=True)

[nltk_data] Downloading package punkt to /Users/nv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /Users/nv/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/nv/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2019-01-22 13:17:02.941 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /Users/nv/dp_dev/lib/python3.6/site-packages/pymorphy2_dicts-2.4.393442.3710985-py3.6.egg/pymorphy2_dicts/data
2019-01-22 13:17:02.999 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2019-01-22 13:17:03.8 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn

This line of code takes existing model and trains it with your data. Once you've trained your model, you can load it without trainig simply writing `faq = FAQSkill(load_path='./faq_doctor')`. In future tutorials you will learn, that `FAQSkill` is a simple wraper on config file, that describes NLP model for intent recognition, and if you want to have more controll under config file, you can pass other parameters, that you want to change in `edit_dict` argument. More information about config files you can find [here](http://docs.deeppavlov.ai/en/master/intro/config_description.html).

The last step is to bring all skills together using Agent.

In [4]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

agent = DefaultAgent([hello, bye, faq, fallback], skills_selector=HighestConfidenceSelector())

Before connecting to Alice, check that everything works correctly. Agent is able to receive a list of messages.

In [5]:
print(agent(['Привет', 'Потерял код регистрации', 'Пока', 'бла бла бла']))

['Приветик', 'Вы можете написать нам письмо на почту приемной кампании, с адреса, который указали при регистрации или обратиться к нашим сотрудниками через Telegram и предоставить информацию, которая позволит идентифицировать вашего ребенка (ФИО, дата рождения и адрес электронной почты, указанный при регистрации).', 'Пока!', 'Я не понял, но могу попробовать ответить на другой вопрос']


/Users/nv/Projects/DeepPavlov/deeppavlov/models/classifiers/cos_sim_classifier.py:81: RuntimeWarning: invalid value encountered in true_divide
  labels_scores = labels_scores/labels_scores.sum(axis=1, keepdims=True)


In order to connect to Alice we need an ssl key and a certificate. Replace `<MACHINE_IP_ADDRESS>` with actual IP address or a domain name of your server.

In [ ]:
!openssl req -new -newkey rsa:4096 -days 365 -nodes -x509 -subj "/CN=<MACHINE_IP_ADDRESS>" -keyout my.key -out my.crt

In [ ]:
from utils.alice import start_agent_server

start_agent_server(agent, host='0.0.0.0', port=5000, endpoint='/faq', ssl_key='my.key', ssl_cert='my.crt')

Now you can specify webhook (`<MACHINE_IP_ADDRESS>:5000/faq`) in [Yandex.Alice skill settings](https://dialogs.yandex.ru/developer/) and test it!


# Useful links

[DeepPavlov repository](https://github.com/deepmipt/DeepPavlov)

[DeepPavlov demo page](https://demo.ipavlov.ai)

[DeepPavlov documentation](https://docs.deeppavlov.ai)

[DeepPavlov blog](https://medium.com/deeppavlov)